# <b>Task:</b> Implement and compare algorithms for finding a substring in a string.

<b>Data</b>

The data archive was used for the evaluation
Structure: 

bad_t_1 - bad set with text

bad_w_1 - bad set with pattern

good_t_1 - good set with text

good_w_1 - good set with pattern


"Bad set" - a set difficult to find, for some algorithms.
"Good set" - a set with plain text.


## Import libraries + reading data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_test_alg.zip to data_test_alg.zip


In [ ]:
!unzip data_test_alg.zip

Archive:  data_test_alg.zip
   creating: benchmarks/
   creating: benchmarks/benchmarks/
  inflating: benchmarks/benchmarks/bad_t_1.txt  
  inflating: benchmarks/benchmarks/bad_t_2.txt  
  inflating: benchmarks/benchmarks/bad_t_3.txt  
  inflating: benchmarks/benchmarks/bad_t_4.txt  
  inflating: benchmarks/benchmarks/bad_w_1.txt  
  inflating: benchmarks/benchmarks/bad_w_2.txt  
  inflating: benchmarks/benchmarks/bad_w_3.txt  
  inflating: benchmarks/benchmarks/bad_w_4.txt  
  inflating: benchmarks/benchmarks/good_t_1.txt  
  inflating: benchmarks/benchmarks/good_t_2.txt  
  inflating: benchmarks/benchmarks/good_t_3.txt  
  inflating: benchmarks/benchmarks/good_t_4.txt  
  inflating: benchmarks/benchmarks/good_w_1.txt  
  inflating: benchmarks/benchmarks/good_w_2.txt  
  inflating: benchmarks/benchmarks/good_w_3.txt  
  inflating: benchmarks/benchmarks/good_w_4.txt  


In [ ]:
import math

def getPrimes(limit):
    primes = []
    numbers = [True] * limit
    for i in range(2, limit):
        if numbers[i]:
            primes.append(i)
            for n in range(i ** 2, limit, i):
                numbers[n] = False
    return primes


def getNerestPrime(number):
  primes = getPrimes(number + 100)
  maxDist = math.inf
  numb = 0

  for p in primes:
      if abs(number - p) < maxDist:
          maxDist = abs(number - p)
          numb = p
  return numb

## Naive algorithm


In [ ]:
def nativeStringMatcher(T, W):
    N = len(T)
    M = len(W)
    count_operation = 0
    for i in range(N - M + 1):
        count_operation += 1
        if T[i:i + M].__eq__(W):
            return i,count_operation
    return -1,count_operation

## Rabin-Karp algorithm

In [ ]:
def RabinKarpNative(T, W):
    hashValue = hash(W)
    count_operation = 0
    for i in range(0, len(T) - 1):
        count_operation += 1
        currValue = hash(T[i: len(W) + i])
        if hashValue == currValue:
            count_operation += 1
            if W.__eq__(T[i:len(W)+i]):
                return i,count_operation
    return -1,count_operation

In [ ]:
def calculateFirstlyHash(string, P):
      hashValue = 0
      i = 0
      for char in string:
          hashValue += ord(char) * (P ** i)
          i += 1
      return hashValue % P, hashValue

In [ ]:
def calculateHashAfterHashing(string, P, previous):
    previous -= ord(string[0])
    previous //= P
    previous += ord(string[-1]) * (P ** (len(string)-2))
    return previous % P, previous

In [ ]:
def RabinKarpProgressive(T, W, P):
    hashValue, l = calculateFirstlyHash(W, P)
    currValue, checksum = calculateFirstlyHash(T[0: len(W)], P)
    count_operation = 1
    if hashValue == currValue:
        if W.__eq__(T[0:len(W)]):
            return 0
    for i in range(1, len(T) - 1):
        currValue, checksum = calculateHashAfterHashing(T[i-1: len(W) + i], P, checksum)
        count_operation += 1
        if hashValue == currValue:
            count_operation += 1
            if W.__eq__(T[i:len(W) + i]):
                return i, count_operation
    return -1, count_operation

## Knutt-Maurice-Pratt algorithm

In [ ]:
def calculatePrefix(W):
    prefixes = [0 for _ in range(len(W))]
    for i in range(1, len(W)):
        offset = prefixes[i-1]
        while offset > 0 and W[offset] != W[i]:
            offset = prefixes[offset-1]
        if W[offset] == W[i]:
            offset = offset + 1
        prefixes[i] = offset
    return prefixes

In [ ]:
def KnutMorisPratt(T, W):
    f = calculatePrefix(W)
    offset = 0
    count_operation = 0
    for i in range(len(T)):

        count_operation += 2
        while offset > 0 and W[offset] != T[i]:
            offset = f[offset-1]
            count_operation += 2

        count_operation += 1
        if W[offset] == T[i]:
            offset = offset + 1

        count_operation += 1
        if offset == len(W):
            return i - len(W) + 1, count_operation

    return -1, count_operation

## Testing

In [ ]:
import time
import numpy as np
def check_time(f, *args):
  function_time = 0.
  for i in np.arange(1000):
    start = time.time()
    f(*args)
    function_time += time.time() - start
  avarege_time = function_time / 1000
  return avarege_time

In [ ]:
def get_text_word(path_text,path_word):
  with open(path_text) as file:
    text = file.read()
  with open(path_word) as file:
    word = file.read()
  return text,word


In [ ]:
text, word = get_text_word("benchmarks/benchmarks/bad_t_1.txt", "benchmarks/benchmarks/bad_w_1.txt")
P = getNerestPrime(len(set(list(word))))

print(nativeStringMatcher(text, word))
print(RabinKarpNative(text, word))
print(RabinKarpProgressive(text,word,P))
print(KnutMorisPratt(text, word))


print(check_time(nativeStringMatcher,text, word))
print(check_time(RabinKarpNative,text, word))
print(check_time(RabinKarpProgressive,text, word, P))
print(check_time(KnutMorisPratt,text, word))


(8, 9)
(8, 10)
(8, 17)
(8, 56)
4.4097900390625e-06
3.958225250244141e-06
1.403045654296875e-05
6.757497787475586e-06


In [ ]:
text, word = get_text_word("benchmarks/benchmarks/bad_t_2.txt", "benchmarks/benchmarks/bad_w_2.txt")
P = getNerestPrime(len(set(list(word))))

print(nativeStringMatcher(text, word))
print(RabinKarpNative(text, word))
print(RabinKarpProgressive(text,word,P))
print(KnutMorisPratt(text, word))


print(check_time(nativeStringMatcher,text, word))
print(check_time(RabinKarpNative,text, word))
print(check_time(RabinKarpProgressive,text, word, P))
print(check_time(KnutMorisPratt,text, word))


(90, 91)
(90, 92)
(90, 181)
(90, 580)
2.466106414794922e-05
3.026270866394043e-05
0.0001371347904205322
6.77950382232666e-05


In [ ]:
text, word = get_text_word("benchmarks/benchmarks/bad_t_3.txt", "benchmarks/benchmarks/bad_w_3.txt")
P = getNerestPrime(len(set(list(word))))

print(nativeStringMatcher(text, word))
print(RabinKarpNative(text, word))
print(RabinKarpProgressive(text,word,P))
print(KnutMorisPratt(text, word))


print(check_time(nativeStringMatcher,text, word))
print(check_time(RabinKarpNative,text, word))
print(check_time(RabinKarpProgressive,text, word, P))
print(check_time(KnutMorisPratt,text, word))


(900, 901)
(900, 902)
(900, 1801)
(900, 5800)
0.00027598190307617185
0.0003782751560211182
0.0017602319717407227
0.0006889259815216065


In [ ]:
text, word = get_text_word("benchmarks/benchmarks/bad_t_4.txt", "benchmarks/benchmarks/bad_w_4.txt")
P = getNerestPrime(len(set(list(word))))

print(nativeStringMatcher(text, word))
print(RabinKarpNative(text, word))
print(RabinKarpProgressive(text,word,P))
print(KnutMorisPratt(text, word))


print(check_time(nativeStringMatcher,text, word))
print(check_time(RabinKarpNative,text, word))
print(check_time(RabinKarpProgressive,text, word, P))
print(check_time(KnutMorisPratt,text, word))

(4000, 4001)
(4000, 4002)
(4000, 8001)
(4000, 28000)
0.0014374380111694335
0.004185657262802124
0.016407800674438477
0.003798530578613281


In [ ]:
text, word = get_text_word("benchmarks/benchmarks/good_t_1.txt", "benchmarks/benchmarks/good_w_1.txt")
P = getNerestPrime(len(set(list(word))))

print(nativeStringMatcher(text, word))
print(RabinKarpNative(text, word))
print(RabinKarpProgressive(text,word,P))
print(KnutMorisPratt(text, word))


print(check_time(nativeStringMatcher,text, word))
print(check_time(RabinKarpNative,text, word))
print(check_time(RabinKarpProgressive,text, word, P))
print(check_time(KnutMorisPratt,text, word))

(599, 600)
(599, 601)
(599, 645)
(599, 2498)
0.0001651732921600342
0.00024545311927795413
0.0008665735721588134
0.00026307034492492674


In [ ]:
text, word = get_text_word("benchmarks/benchmarks/good_t_2.txt", "benchmarks/benchmarks/good_w_2.txt")
P = getNerestPrime(len(set(list(word))))

print(nativeStringMatcher(text, word))
print(RabinKarpNative(text, word))
print(RabinKarpProgressive(text,word,P))
print(KnutMorisPratt(text, word))


print(check_time(nativeStringMatcher,text, word))
print(check_time(RabinKarpNative,text, word))
print(check_time(RabinKarpProgressive,text, word, P))
print(check_time(KnutMorisPratt,text, word))


(610, 611)
(610, 612)
(610, 744)
(610, 2780)
0.00018070554733276366
0.00033298873901367186
0.0013962843418121337
0.00031975555419921876


In [ ]:
text, word = get_text_word("benchmarks/benchmarks/good_t_3.txt", "benchmarks/benchmarks/good_w_3.txt")
P = getNerestPrime(len(set(list(word))))

print(nativeStringMatcher(text, word))
print(RabinKarpNative(text, word))
print(RabinKarpProgressive(text,word,P))
print(KnutMorisPratt(text, word))


print(check_time(nativeStringMatcher,text, word))
print(check_time(RabinKarpNative,text, word))
print(check_time(RabinKarpProgressive,text, word, P))
print(check_time(KnutMorisPratt,text, word))

(1629, 1630)
(1629, 1631)
(1629, 1687)
(1629, 8152)
0.0005670225620269775
0.0014754645824432372
0.009586340188980103
0.0010820918083190919


In [ ]:
text, word = get_text_word("benchmarks/benchmarks/good_t_4.txt", "benchmarks/benchmarks/good_w_4.txt")
P = getNerestPrime(len(set(list(word))))

print(nativeStringMatcher(text, word))
print(RabinKarpNative(text, word))
print(RabinKarpProgressive(text,word,P))
print(KnutMorisPratt(text, word))


print(check_time(nativeStringMatcher,text, word))
print(check_time(RabinKarpNative,text, word))
print(check_time(RabinKarpProgressive,text, word, P))
print(check_time(KnutMorisPratt,text, word))

(9522, 9523)
(9522, 9524)
(9522, 9714)
(9522, 38456)
0.00308068323135376
0.0047944443225860596
0.02156748652458191
0.004129459857940674
